In [31]:
import os
import json
from zipfile import ZipFile
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers,models

In [32]:
kaggle_credentails = json.load(open("kaggle.json"))

In [33]:
os.environ['KAGGLE_USERNAME'] = kaggle_credentails["username"]
os.environ['KAGGLE_KEY'] = kaggle_credentails["key"]

In [ ]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset

Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0


In [ ]:
with ZipFile ('plantvillage-dataset.zip','r') as zip_ref:
    zip_ref.extractall()

In [ ]:
print(os.listdir("plantvillage dataset"))

print(len(os.listdir("plantvillage dataset/segmented")))
print(os.listdir("plantvillage dataset/segmented")[:5])

print(len(os.listdir("plantvillage dataset/color")))
print(os.listdir("plantvillage dataset/color")[:5])

print(len(os.listdir("plantvillage dataset/grayscale")))
print(os.listdir("plantvillage dataset/grayscale")[:5])

In [ ]:
base_dir = 'plantvillage dataset/color'

In [ ]:
image_path ='plantvillage dataset/color/Apple___Apple_scab/ff99efdc-a9f8-4360-9c64-f8274f456be5___FREC_Scab 3161.JPG'

img = mpimg.imread(image_path)

print(img.shape)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:

imd_size =224
batch_size = 32

In [ ]:
data_gen = ImageDataGenerator(rescale=1./225,
                             validation_split=0.2
                             )

In [ ]:
train_generator= data_gen.flow_from_directory(
    base_dir,
    target_size=(imd_size,imd_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'

)

In [ ]:
validation_generator= data_gen.flow_from_directory(
    base_dir,
    target_size=(imd_size,imd_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical')

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=(imd_size,imd_size,3)))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(256 , activation='relu'))
model.add(layers.Dense(train_generator.num_classes, activation="softmax"))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10
)

In [ ]:
print("Evalutaing model....")
val_loss , val_accuracy = model.evaluate(validation_generator,steps = validation_generator.samples // batch_size)
print(f"Validation accuracy : {val_accuracy*100:.2f}%")

In [ ]:
def load_and_preprocess_image(image_path , target_size=(224 , 224)):
    img = Image.open(image_path)
    img = img.resize((target_size))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array , axis=0)
    img_array = img_array.astype('float32') / 255.0
    return img_array

def predict_image_class(image_path , model ,class_indices):
    preprocessed_img = load_and_preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(prediction , axis=1)[0]
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name

In [ ]:
class_indices = {v:k for k,v in train_generator.class_indices.items()}

In [ ]:
class_indices

In [ ]:
json.dump(class_indices,open("class_indices.json","w"))

In [ ]:
image_path = '/content/new.webp'

predicted_class_name = predict_image_class(image_path , model , class_indices)
print(f"The predicted class for the image is: {predicted_class_name}")

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
model_save_path = "/content/drive/MyDrive/trained_models/plant_disease_model.h5"
model_dir = os.path.dirname(model_save_path)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
model.save(model_save_path)